In [1]:
import requests
import pandas as pd
from json import loads

### User Inputs

In [2]:
user = "Filpill"

### User Overview

In [3]:
url = f"https://www.codewars.com/api/v1/users/{user}"
response = requests.get(url)
json_data = loads(response.text)
print("Status:",response.status_code)
curated_cols = ['username','honor','leaderboardPosition','ranks.overall.name','ranks.languages.python.score','ranks.languages.sql.score','ranks.overall.score','codeChallenges.totalCompleted']
df_profile = pd.json_normalize(json_data)
profile_cols = list(df_profile.columns)
df_profile = df_profile[df_profile.columns.intersection(curated_cols)]
df_profile.rename(columns={'leaderboardPosition':'leaderboard',
                           'ranks.overall.name':'overall_rank',
                           'ranks.languages.python.score':'python_score',
                           'ranks.languages.sql.score':'sql_score',
                           'ranks.overall.score':'total_score',
                           'codeChallenges.totalCompleted':'total_completed'},inplace=True)
df_profile

Status: 200


,username,honor,leaderboard,overall_rank,total_score,python_score,sql_score,total_completed
0,Filpill,983,24579,4 kyu,1350,868,482,85


### List of Completed Challenges

In [4]:
page = 0
url = f"https://www.codewars.com/api/v1/users/{user}/code-challenges/completed?page={page}"
response = requests.get(url)
json_data = loads(response.text)
print("Status:",response.status_code)
raw_data = pd.json_normalize(json_data)
df_complete = pd.DataFrame.from_dict(raw_data.loc[0,'data'])
completed_list = list(df_complete['id'])
df_complete

Status: 200


,id,name,slug,completedLanguages,completedAt
0,521c2db8ddc89b9b7a0000c1,Snail,snail,[python],2022-11-09T11:53:41.303Z
1,546d15cebed2e10334000ed9,Find the unknown digit,find-the-unknown-digit,[python],2022-11-07T18:42:10.416Z
2,55c6126177c9441a570000cc,Weight for weight,weight-for-weight,[python],2022-11-07T00:10:56.623Z
3,52bc74d4ac05d0945d00054e,First non-repeating character,first-non-repeating-character,[python],2022-11-06T21:05:31.722Z
4,5541f58a944b85ce6d00006a,Product of consecutive Fib numbers,product-of-consecutive-fib-numbers,[python],2022-11-06T19:48:05.497Z
...,...,...,...,...,...
80,55bf01e5a717a0d57e0000ec,Persistent Bugger.,persistent-bugger,[python],2022-06-18T12:33:42.290Z
81,55b42574ff091733d900002f,Friend or Foe?,friend-or-foe,[python],2022-06-18T11:30:41.704Z
82,5552101f47fc5178b1000050,Playing with digits,playing-with-digits,[python],2022-06-18T11:21:50.614Z
83,525f50e3b73515a6db000b83,Create Phone Number,create-phone-number,[python],2022-06-18T10:27:17.433Z


### Getting Details of Code Challenges Completed By User

In [13]:
url = f"https://www.codewars.com/api/v1/code-challenges/{completed_list[0]}"
response = requests.get(url)
json_data = loads(response.text)
details_cols =  list(pd.json_normalize(json_data).columns)
df_kata = pd.DataFrame(columns=details_cols) #Making empty dataframe with relevant columns

#Looping through all the completed challenge id's
for kata_id in completed_list:
    url = f"https://www.codewars.com/api/v1/code-challenges/{kata_id}"
    response = requests.get(url)
    json_data = loads(response.text)
    row = pd.json_normalize(json_data)
    df_kata = pd.concat([df_kata,row]) 
df_kata.head(2)

,id,name,slug,category,publishedAt,approvedAt,languages,url,createdAt,description,...,contributorsWanted,rank.id,rank.name,rank.color,createdBy.username,createdBy.url,unresolved.issues,unresolved.suggestions,approvedBy.username,approvedBy.url
0,521c2db8ddc89b9b7a0000c1,Snail,snail,algorithms,2013-08-28T04:56:02.580Z,None,"[ruby, coffeescript, javascript, python, haske...",https://www.codewars.com/kata/521c2db8ddc89b9b...,2013-08-27T04:40:24.711Z,"## Snail Sort\n\nGiven an `n x n` array, retur...",...,True,-4,4 kyu,blue,stevenbarragan,https://www.codewars.com/users/stevenbarragan,6,6,NaN,NaN
0,546d15cebed2e10334000ed9,Find the unknown digit,find-the-unknown-digit,games,2014-11-20T05:25:08.613Z,2016-11-05T15:00:55.672Z,"[java, csharp, javascript, python, php, groovy...",https://www.codewars.com/kata/546d15cebed2e103...,2014-11-19T22:12:30.057Z,To give credit where credit is due: This probl...,...,True,-4,4 kyu,blue,JJ-Bug,https://www.codewars.com/users/JJ-Bug,5,11,user5036852,https://www.codewars.com/users/user5036852


### Splitting Out The Details Into Lists

In [16]:
challenge_details = list(df_kata.columns)
selected_details = ['id','totalAttempts','totalCompleted','voteScore','rank.name','rank.color','category','tags']
print(challenge_details)
df_details_select = df_kata[df_kata.columns.intersection(selected_details)]
df_details_select

['id', 'name', 'slug', 'category', 'publishedAt', 'approvedAt', 'languages', 'url', 'createdAt', 'description', 'totalAttempts', 'totalCompleted', 'totalStars', 'voteScore', 'tags', 'contributorsWanted', 'rank.id', 'rank.name', 'rank.color', 'createdBy.username', 'createdBy.url', 'unresolved.issues', 'unresolved.suggestions', 'approvedBy.username', 'approvedBy.url']


,id,category,totalAttempts,totalCompleted,voteScore,tags,rank.name,rank.color
0,521c2db8ddc89b9b7a0000c1,algorithms,400320,53691,5860,"[Arrays, Algorithms]",4 kyu,blue
0,546d15cebed2e10334000ed9,games,84453,5868,762,"[Mathematics, Algorithms, Puzzles]",4 kyu,blue
0,55c6126177c9441a570000cc,algorithms,310710,41248,3492,[Algorithms],5 kyu,yellow
0,52bc74d4ac05d0945d00054e,algorithms,287803,45533,3445,"[Strings, Algorithms]",5 kyu,yellow
0,5541f58a944b85ce6d00006a,reference,182643,49136,3878,"[Algorithms, Mathematics]",5 kyu,yellow
...,...,...,...,...,...,...,...,...
0,55bf01e5a717a0d57e0000ec,reference,502562,140823,12234,[Fundamentals],6 kyu,yellow
0,55b42574ff091733d900002f,reference,278325,124487,9274,[Fundamentals],7 kyu,white
0,5552101f47fc5178b1000050,reference,394996,102214,9282,"[Fundamentals, Mathematics]",6 kyu,yellow
0,525f50e3b73515a6db000b83,algorithms,555909,229850,15483,"[Arrays, Strings, Regular Expressions, Algorit...",6 kyu,yellow


### Merging Details onto Completitons via the Kata ID

In [17]:
df_kata_combined = pd.merge(df_complete,df_details_select,on='id',how='left')
df_kata_combined

,id,name,slug,completedLanguages,completedAt,category,totalAttempts,totalCompleted,voteScore,tags,rank.name,rank.color
0,521c2db8ddc89b9b7a0000c1,Snail,snail,[python],2022-11-09T11:53:41.303Z,algorithms,400320,53691,5860,"[Arrays, Algorithms]",4 kyu,blue
1,546d15cebed2e10334000ed9,Find the unknown digit,find-the-unknown-digit,[python],2022-11-07T18:42:10.416Z,games,84453,5868,762,"[Mathematics, Algorithms, Puzzles]",4 kyu,blue
2,55c6126177c9441a570000cc,Weight for weight,weight-for-weight,[python],2022-11-07T00:10:56.623Z,algorithms,310710,41248,3492,[Algorithms],5 kyu,yellow
3,52bc74d4ac05d0945d00054e,First non-repeating character,first-non-repeating-character,[python],2022-11-06T21:05:31.722Z,algorithms,287803,45533,3445,"[Strings, Algorithms]",5 kyu,yellow
4,5541f58a944b85ce6d00006a,Product of consecutive Fib numbers,product-of-consecutive-fib-numbers,[python],2022-11-06T19:48:05.497Z,reference,182643,49136,3878,"[Algorithms, Mathematics]",5 kyu,yellow
...,...,...,...,...,...,...,...,...,...,...,...,...
80,55bf01e5a717a0d57e0000ec,Persistent Bugger.,persistent-bugger,[python],2022-06-18T12:33:42.290Z,reference,502562,140823,12234,[Fundamentals],6 kyu,yellow
81,55b42574ff091733d900002f,Friend or Foe?,friend-or-foe,[python],2022-06-18T11:30:41.704Z,reference,278325,124487,9274,[Fundamentals],7 kyu,white
82,5552101f47fc5178b1000050,Playing with digits,playing-with-digits,[python],2022-06-18T11:21:50.614Z,reference,394996,102214,9282,"[Fundamentals, Mathematics]",6 kyu,yellow
83,525f50e3b73515a6db000b83,Create Phone Number,create-phone-number,[python],2022-06-18T10:27:17.433Z,algorithms,555909,229850,15483,"[Arrays, Strings, Regular Expressions, Algorit...",6 kyu,yellow
